In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os

strategy = tf.distribute.MirroredStrategy()

print("Number of accelerators: ", strategy.num_replicas_in_sync)

2022-05-03 11:35:47.052117: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 11:35:47.778436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30970 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1


In [2]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train = "./data/real_vs_fake/real-vs-fake/train"
valid = "./data/real_vs_fake/real-vs-fake/valid"
test = "./data/real_vs_fake/real-vs-fake/test"

train_generator1 = train_datagen.flow_from_directory(train,
                                                     batch_size=64,
                                                     target_size=(128, 128),
                                                     class_mode='binary',
                                                     )

val_generator = train_datagen.flow_from_directory(valid,
                                                     batch_size=32,
                                                     target_size=(128, 128),
                                                     class_mode='binary',
                                                     )

test_generator = train_datagen.flow_from_directory(test,
                                                     batch_size=64,
                                                     target_size=(128, 128),
                                                     class_mode='binary',
                                                     )

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
train_generator1.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [6]:
transfer1 = tf.keras.applications.DenseNet121(
  weights='imagenet',
  input_shape=(128, 128, 3),
  include_top=False
)

transfer2 = tf.keras.applications.ResNet101(
  weights='imagenet',
  input_shape=(128, 128, 3),
  include_top=False
)

transfer3 = tf.keras.applications.MobileNetV2(
  weights='imagenet',
  input_shape=(128, 128, 3),
  include_top=False
)

model1 = Sequential([
  transfer1,
  GlobalAveragePooling2D(),
  Dense(256, activation='relu'),
  BatchNormalization(),
  Dropout(0.5),
  Dense(2, activation='softmax')
])

model2 = Sequential([
  transfer2,
  GlobalAveragePooling2D(),
  Dense(256, activation='relu'),
  BatchNormalization(),
  Dropout(0.5),
  Dense(2, activation='softmax')
])

model3 = Sequential([
  transfer3,
  GlobalAveragePooling2D(),
  Dense(256, activation='relu'),
  BatchNormalization(),
  Dropout(0.5),
  Dense(2, activation='softmax')
])

model4 = Sequential([
  Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
  BatchNormalization(),
  Conv2D(32, kernel_size=(3, 3), activation='relu'),
  BatchNormalization(),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(64, kernel_size=(3, 3), activation='relu'),
  BatchNormalization(),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  BatchNormalization(),
  Dropout(0.25),
  Dense(2, activation='softmax')
])

models = [model1, model2, model3, model4]
model_input = tf.keras.layers.Input(shape=(128, 128, 3))
model_outputs = [model(model_input) for model in models]
ensemble_op = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_op)

ensemble_model.summary()

9420800/9406464 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 2)            7301442     input_4[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2)            43184258    input_4[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 2)            2587458     input_4[0][0]                    
_______________________________

In [7]:
ensemble_model.compile(optimizer=Adam(learning_rate=0.001),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

In [8]:
def schedular(epoch):
  if epoch <= 2:
    return 0.001
  elif epoch > 2 and epoch <= 15:
    return 0.0001
  else:
    return 0.00001
  
lr_callbacks = tf.keras.callbacks.LearningRateScheduler(schedular)

In [9]:
hist = ensemble_model.fit(train_generator1, steps_per_epoch=100000//64, validation_steps=20000//64, epochs=20, callbacks=[lr_callbacks], validation_data=val_generator)

2022-05-03 11:50:50.857846: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-05-03 11:51:12.875200: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


1562/1562 [==============================] - 498s 302ms/step - loss: 0.3734 - accuracy: 0.8426 - val_loss: 0.5198 - val_accuracy: 0.7437
Epoch 2/20
1562/1562 [==============================] - 465s 297ms/step - loss: 0.2360 - accuracy: 0.9216 - val_loss: 0.3813 - val_accuracy: 0.8528
Epoch 3/20
1562/1562 [==============================] - 466s 298ms/step - loss: 0.1931 - accuracy: 0.9417 - val_loss: 0.3529 - val_accuracy: 0.8755
Epoch 4/20
1562/1562 [==============================] - 466s 298ms/step - loss: 0.1168 - accuracy: 0.9700 - val_loss: 0.1269 - val_accuracy: 0.9652
Epoch 5/20
1562/1562 [==============================] - 466s 298ms/step - loss: 0.0926 - accuracy: 0.9787 - val_loss: 0.0948 - val_accuracy: 0.9790
Epoch 6/20
1562/1562 [==============================] - 466s 298ms/step - loss: 0.0836 - accuracy: 0.9821 - val_loss: 0.0932 - val_accuracy: 0.9822
Epoch 7/20
1562/1562 [==============================] - 466s 298ms/step - loss: 0.0724 - accuracy: 0.9860 - val_loss: 0.090